In [185]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import os

for dirname, _, filenames in os.walk('../data/raw'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../data/raw/sales_test.csv
../data/raw/solution.csv
../data/raw/sales_train.csv
../data/raw/calendar.csv
../data/raw/test_weights.csv
../data/raw/inventory.csv


In [186]:
calendar = pd.read_csv('../data/raw/calendar.csv')
test_weights = pd.read_csv('../data/raw/test_weights.csv')
inventory = pd.read_csv('../data/raw/inventory.csv')
sales_train = pd.read_csv('../data/raw/sales_train.csv')
sales_test = pd.read_csv('../data/raw/sales_test.csv')
solution = pd.read_csv('../data/raw/solution.csv')

In [187]:
def combine_sales_data(target_df, inventory_df, calendar_df):
    """
    Combines sales data (target_df) with inventory and calendar data, modifying the target_df in place.
    
    Parameters:
    target_df (pd.DataFrame): The sales data (either from sales_train or sales_test), which will be modified.
    inventory_df (pd.DataFrame): The inventory data containing product details.
    calendar_df (pd.DataFrame): The calendar data containing information about holidays and events.
    """

    
    # Merge target_df with inventory_df on 'unique_id'
    target_df = target_df.merge(inventory_df[['unique_id', 'product_unique_id', 'name', 
                                  'L1_category_name_en', 'L2_category_name_en', 
                                  'L3_category_name_en', 'L4_category_name_en']], 
                    on='unique_id', how='left')
    
    # Ensure the 'date' column in target_df is in datetime format
    #target_df['date'] = pd.to_datetime(target_df['date'])  # Ensure the date is in datetime format
    
    # Merge target_df with calendar_df on 'date' and 'warehouse'
    target_df = target_df.merge(calendar_df[['date', 'warehouse', 'holiday', 'holiday_name', 
                                 'shops_closed', 'winter_school_holidays', 'school_holidays']], 
                    on=['date', 'warehouse'], how='left')

    return target_df



In [188]:
sales_train = combine_sales_data(sales_train, inventory, calendar)
sales_test = combine_sales_data(sales_test, inventory, calendar)

In [189]:
sales_train['date'] = pd.to_datetime(sales_train['date'])
sales_test['date'] = pd.to_datetime(sales_test['date'])

In [190]:
sales_train.head()

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday,holiday_name,shops_closed,winter_school_holidays,school_holidays
0,4845,2024-03-10,Budapest_1,6436.0,16.34,646.26,1.00,0.00000,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0
1,4845,2021-05-25,Budapest_1,4663.0,12.63,455.96,1.00,0.00000,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0
2,4845,2021-12-20,Budapest_1,6507.0,34.55,455.96,1.00,0.00000,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0
3,4845,2023-04-29,Budapest_1,5463.0,34.52,646.26,0.96,0.20024,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0
4,4845,2022-04-01,Budapest_1,5997.0,35.92,486.41,1.00,0.00000,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0


In [191]:
sales_train.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007419 entries, 0 to 4007418
Data columns (total 25 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   unique_id               4007419 non-null  int64         
 1   date                    4007419 non-null  datetime64[ns]
 2   warehouse               4007419 non-null  object        
 3   total_orders            4007367 non-null  float64       
 4   sales                   4007367 non-null  float64       
 5   sell_price_main         4007419 non-null  float64       
 6   availability            4007419 non-null  float64       
 7   type_0_discount         4007419 non-null  float64       
 8   type_1_discount         4007419 non-null  float64       
 9   type_2_discount         4007419 non-null  float64       
 10  type_3_discount         4007419 non-null  float64       
 11  type_4_discount         4007419 non-null  float64       
 12  type_5_discoun

### Preprocessing

In [192]:
sales_train.drop(['name', 'holiday_name'], axis=1, inplace=True)

sales_train['month'] = sales_train['date'].dt.month
sales_train['weekday'] = sales_train['date'].dt.weekday
sales_train['year'] = sales_train['date'].dt.year

sales_train['unique_date'] = sales_train['unique_id'].astype(str) + '_' + sales_train['date'].astype(str)

sales_train.set_index('unique_date', inplace=True)

sales_train.drop(['date', 'unique_id', 'product_unique_id', 'availability'], axis=1, inplace=True)

sales_train.dropna(inplace=True)

In [193]:
sales_test.drop(['name', 'holiday_name'], axis=1, inplace=True)

sales_test['month'] = sales_test['date'].dt.month
sales_test['weekday'] = sales_test['date'].dt.weekday
sales_test['year'] = sales_test['date'].dt.year

sales_test['unique_date'] = sales_test['unique_id'].astype(str) + '_' + sales_test['date'].astype(str)

sales_test.set_index('unique_date', inplace=True)

sales_test.drop(['date', 'unique_id', 'product_unique_id'], axis=1, inplace=True)

sales_test.dropna(inplace=True)

In [194]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4007367 entries, 4845_2024-03-10 to 4941_2023-06-20
Data columns (total 22 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   warehouse               object 
 1   total_orders            float64
 2   sales                   float64
 3   sell_price_main         float64
 4   type_0_discount         float64
 5   type_1_discount         float64
 6   type_2_discount         float64
 7   type_3_discount         float64
 8   type_4_discount         float64
 9   type_5_discount         float64
 10  type_6_discount         float64
 11  L1_category_name_en     object 
 12  L2_category_name_en     object 
 13  L3_category_name_en     object 
 14  L4_category_name_en     object 
 15  holiday                 int64  
 16  shops_closed            int64  
 17  winter_school_holidays  int64  
 18  school_holidays         int64  
 19  month                   int32  
 20  weekday                 int32  
 21  year          

In [195]:
# Identifier les variables numériques
numerical_cols = sales_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols = [col for col in numerical_cols if col != 'sales']

# Identifier les cardinalités des variables catégorielles
cardinalities = {col: sales_train[col].nunique() for col in sales_train.select_dtypes(include=['object','category']).columns}

# Séparer les colonnes catégorielles selon leur cardinalité
low_cardinality_cols = [col for col, card in cardinalities.items() if card <= 10]
high_cardinality_cols = [col for col, card in cardinalities.items() if 10 < card ]

print(f'Numerical columns: {numerical_cols}')
print(f'Low cardinality columns: {low_cardinality_cols}')
print(f'High cardinality columns: {high_cardinality_cols}')

Numerical columns: ['total_orders', 'sell_price_main', 'type_0_discount', 'type_1_discount', 'type_2_discount', 'type_3_discount', 'type_4_discount', 'type_5_discount', 'type_6_discount', 'holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays']
Low cardinality columns: ['warehouse', 'L1_category_name_en']
High cardinality columns: ['L2_category_name_en', 'L3_category_name_en', 'L4_category_name_en']


In [196]:
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

target = 'sales'

# Preprocessing for numerical data
numerical_transformer = StandardScaler()

# Preprocessing for categorical data for target encoding
categorical_transformer_target = ce.TargetEncoder()

# Preprocessing for categorical data for one-hot encoding
categorical_transformer_one_hot = OneHotEncoder(handle_unknown='ignore')

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat_target', categorical_transformer_target, high_cardinality_cols),
        ('cat_onehot', categorical_transformer_one_hot, low_cardinality_cols)
    ]
)

### Modelling

In [197]:
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

model = xgb.XGBRegressor(
    objective='reg:squarederror', 
    n_estimators=200, 
    max_depth=9, 
    learning_rate=0.15,  # Rounded for simplicity
    subsample=1, 
    colsample_bytree=0.75, 
    gamma=2.5, 
    reg_alpha=0, 
    reg_lambda=100
)

# Create and configure the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Séparer les données en entraînement et test
X = sales_train.drop('sales', axis=1)  
y = sales_train['sales']  

# Diviser en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cross-validation
scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print("Custom scores for each fold:", -scores)
print("Average custom score:", np.sqrt(np.mean(-scores)))

# Train the model on the entire dataset
pipeline.fit(X_train, y_train)


Custom scores for each fold: [13876.59122161 14080.51385038 13970.80800174 14481.26376348
 14006.49645662]
Average custom score: 118.6723837241288


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['total_orders',
                                                   'sell_price_main',
                                                   'type_0_discount',
                                                   'type_1_discount',
                                                   'type_2_discount',
                                                   'type_3_discount',
                                                   'type_4_discount',
                                                   'type_5_discount',
                                                   'type_6_discount', 'holiday',
                                                   'shops_closed',
                                                   'winter_school_holidays',
                                                   'school_holidays']),
                                                 ('cat_target', TargetEncoder(),
                                                  ['...
                              feature_types=None, gamma=2.5, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.15,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=9, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=200, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [198]:
# Faire des prédictions sur l'ensemble de test
y_pred = pipeline.predict(X_test)

# Calculer la MAE
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 51.49429247470879


In [199]:
X_train = X
Y_train = y
X_test = sales_test

# Faire des prédictions sur l'ensemble de test de Rohlik
y_pred = pipeline.predict(X_test)

# Créer un DataFrame avec les prédictions
submission = pd.DataFrame({'id': X_test.index, 'sales': y_pred})

# Enregistrer les prédictions dans un fichier CSV
submission.to_csv('../data/processed/submission.csv', index=False)